In [6]:
!pip install mercury --quiet
## ignore errors

In [7]:
import mercury as mr

# BQKE Helper

## Classes for consuming Knowledge Engine API

In [34]:
"""
  ------------------------------------------
  Classes for consuming Knowledge Engine API
  ------------------------------------------
"""

from datetime import datetime
from typing import List, Optional, Any, Dict, ClassVar
from uuid import UUID

from pydantic import BaseModel, Field, ValidationError
from enum import Enum

"""
  Generic Models for KE Scans API
"""
class Data(BaseModel):
    """Represents the data source for the scan."""
    RESOURCE_TYPE_TABLE: ClassVar[str] = "table"
    RESOURCE_TYPE_DATASET: ClassVar[str] = "dataset"

    resource: str

    @property
    def is_for_table(self) -> bool:
        return self.resource.split('/')[-2][:-1] == self.RESOURCE_TYPE_TABLE

    @property
    def is_for_dataset(self) -> bool:
        return self.resource.split('/')[-2][:-1] == self.RESOURCE_TYPE_DATASET


class OnDemand(BaseModel):
    """Represents an on-demand trigger configuration. Empty in the provided data."""
    pass


class Trigger(BaseModel):
    """Represents the trigger mechanism for a scan."""
    on_demand: OnDemand = Field(..., alias='onDemand')


class ExecutionSpec(BaseModel):
    """Represents the execution specification for a scan."""
    trigger: Trigger


class ExecutionStatus(BaseModel):
    """Represents the execution status of the latest job for a scan."""
    # This field is optional as it's not present in all scan types (e.g., KNOWLEDGE_ENGINE).
    latest_job_start_time: Optional[datetime] = Field(None, alias='latestJobStartTime')
    latest_job_end_time: datetime = Field(..., alias='latestJobEndTime')
    latest_job_create_time: datetime = Field(..., alias='latestJobCreateTime')


class ScanTypeValue(Enum):
    KNOWLEDGE_ENGINE = "KNOWLEDGE_ENGINE"
    DATA_DOCUMENTATION = "DATA_DOCUMENTATION"
    DATA_PROFILE = "DATA_PROFILE"


class DataScan(BaseModel):
    """Represents a single data scan item."""
    name: str
    uid: UUID
    description: Optional[str] = None
    display_name: Optional[str] = Field(None, alias='displayName') #***
    state: str
    create_time: datetime = Field(..., alias='createTime')
    update_time: datetime = Field(..., alias='updateTime')
    data: Data
    execution_spec: ExecutionSpec = Field(..., alias='executionSpec')
    execution_status: ExecutionStatus = Field(..., alias='executionStatus')
    type: ScanTypeValue

    @property
    def is_for_table(self) -> bool:
        return self.data.is_for_table

    @property
    def is_for_dataset(self) -> bool:
        return self.data.is_for_dataset

    @property
    def resource_name(self) -> str:
        return self.data.resource


class DataScansResponse(BaseModel):
    """The root model for the entire JSON API response."""
    data_scans: List[DataScan] = Field(..., alias='dataScans')


"""
  type KNOWLEDGE_ENGINE models
"""
class KESpec(BaseModel):
    """Represents knowledgeEngineSpec."""
    pass


class ColumnTuple(BaseModel):
    """Represents a fully qualified column used in a join relationship."""
    entry_fqn: str = Field(..., alias='entryFqn', description="Fully qualified name of the BigQuery table.")
    field_path: str = Field(..., alias='fieldPath', description="The name of the column.")


class SchemaRelationship(BaseModel):
    """Defines a join relationship between two sets of columns."""
    left_columns_tuple: List[ColumnTuple] = Field(..., alias='leftColumnsTuple')
    right_columns_tuple: List[ColumnTuple] = Field(..., alias='rightColumnsTuple')
    type: str = Field(..., description="The type of relationship, e.g., 'JOIN'.")


class BusinessTerm(BaseModel):
    """A single term and its definition from the business glossary."""
    title: str
    description: str


class BusinessGlossary(BaseModel):
    """Contains a list of business terms relevant to the dataset."""
    terms: List[BusinessTerm]


class DatasetResult(BaseModel):
    """Contains the description, schema relationships, and glossary for a dataset."""
    description: str
    schema_relationship: List[SchemaRelationship] = Field(..., alias='schemaRelationship')
    business_glossary: BusinessGlossary = Field(..., alias='businessGlossary')


class KEResult(BaseModel):
    """The main result object from a KNOWLEDGE_ENGINE data scan."""
    dataset_result: DatasetResult = Field(..., alias='datasetResult')


class KEScan(DataScan):
    """Represents a KNOWLEDGE_ENGINE data scan."""
    knowledge_engine_spec: Optional[KESpec] = Field(None, alias='knowledgeEngineSpec')
    knowledge_engine_result: KEResult = Field(..., alias='knowledgeEngineResult')

    @property
    def dataset_description(self) -> str:
        return self.knowledge_engine_result.dataset_result.description # shortcut

    @property
    def business_glossary(self) -> BusinessGlossary:
        return self.knowledge_engine_result.dataset_result.business_glossary

    @property
    def schema_relationships(self) -> SchemaRelationship:
        return self.knowledge_engine_result.dataset_result.schema_relationship

"""
  type DATA_DOCUMENTATION generic models
"""

class DDSpec(BaseModel):
    """Represents dataDocumentationSpec."""
    pass


class Query(BaseModel):
    """Represents a single SQL query with its description."""
    sql: str
    description: str



"""
  type DATA_DOCUMENTATION table models
"""
class SchemaField(BaseModel):
    """Represents a single field (column) in a table schema."""
    name: str
    description: str


class Schema(BaseModel):
    """Represents the schema of a table, containing a list of fields."""
    fields: List[SchemaField]


class TableResult(BaseModel):
    """Contains the detailed documentation results for a specific table."""
    overview: str
    the_schema: Schema = Field(alias="schema") # renamed to the_schema to preven collision
    queries: List[Query]
    query_theme: Optional[Dict[str, Any]] = Field(None, alias='queryTheme')


class DDTableResult(BaseModel):
    """The main result object from a DATA_DOCUMENTATION table scan."""
    queries: List[Query]
    overview: str
    the_schema: Schema = Field(alias="schema") # renamed to the_schema to preven collision
    table_result: TableResult = Field(..., alias='tableResult')


class DDTableScan(DataScan):
    """Represents a DATA_DOCUMENTATION data scan."""
    data_documentation_spec: Optional[DDSpec] = Field(None, alias='dataDocumentationSpec')
    data_documentation_result: DDTableResult = Field(..., alias='dataDocumentationResult')

    @property
    def full_table_name(self) -> str:
        parts = self.data.resource.split('/')
        return f"{parts[4]}.{parts[6]}.{parts[8]}"

    @property
    def overview(self) -> str:
        return self.data_documentation_result.table_result.overview # shortcut

    @property
    def fields(self) -> List[SchemaField]:
        return self.data_documentation_result.table_result.the_schema.fields

    @property
    def queries(self) -> List[Query]:
        return self.data_documentation_result.table_result.queries

"""
  type DATA_DOCUMENTATION dataset models
"""
class DDDatasetResult(BaseModel):
    queries: List[Query]

class DDDataDocumentationResult(BaseModel):
    """The main result object from a DATA_DOCUMENTATION dataset scan."""
    queries: List[Query]
    dataset_result: DDDatasetResult = Field(..., alias='datasetResult')


class DDDatasetScan(DataScan):
    """Represents a DATA_DOCUMENTATION dataset scan."""
    data_documentation_spec: Optional[DDSpec] = Field(None, alias='dataDocumentationSpec')
    data_documentation_result: DDDataDocumentationResult = Field(..., alias='dataDocumentationResult')

    @property
    def queries(self) -> List[Query]:
        return self.data_documentation_result.dataset_result.queries



## KEDataScanHelper

### Classes for output from KEDatasetScanHelper

In [35]:
"""
  ------------------------------------------
  Classes for output from KEDatasetScanHelper
  ------------------------------------------
"""
import json

class KEDatasetTable(BaseModel):
    """
    Represents a single table.
    """
    name: str
    overview: Optional[str] = None
    fields: List[SchemaField] = Field(..., description="A list of fields in the table.")
    queries: List[Query] = Field(..., description="A list of queries that can be run against the table.")
    ddl: Optional[str] = None
    row_count: Optional[int] = None
    size_bytes: Optional[int] = None

    @property
    def fields_json(self) -> str:
        full_model = self.model_dump()
        return json.dumps(full_model['fields'])

    @property
    def queries_json(self) -> str:
        full_model = self.model_dump()
        return json.dumps(full_model['queries'])

    @property
    def text_field_descriptions(self) -> str:
        field_descriptions = '```\n'
        for field in self.fields:
            field_descriptions += f"`{field.name}` -- Definition: {field.description}\n"

        field_descriptions += '```'

        return field_descriptions


class KEDatasetRelationship(BaseModel):
    """
    Represents a single relationship between two database tables.
    """
    table1: str = Field(..., description="The name of the first table in the relationship.")
    table2: str = Field(..., description="The name of the second table in the relationship.")
    relationship: str = Field(..., description="The join condition that defines the relationship.")
    source: str = Field(..., description="The source that inferred or defined this relationship.")

class KEDatasetDetails(BaseModel):
    """
    Represents the detailed documentation results for a specific dataset.
    """
    project_id: str = Field(..., description="Project ID of the dataset.")
    dataset_name: str = Field(..., description="Name of the dataset")
    dataset_location: str = Field(..., description="Location of the dataset.")
    dataset_description: str = Field(..., description="A brief overview of the dataset.")
    dataset_relationships: List[KEDatasetRelationship] = Field(..., description="A list of table relationships.")
    dataset_queries: List[Query] = Field(..., description="A list of queries that can be run against the dataset.")
    dataset_business_glossary: List[BusinessTerm] = Field(..., description="A list of business glossary terms.")
    dataset_tables: List[KEDatasetTable] = Field(..., description="A list of tables in the dataset.")

    @property
    def dataset_relationships_json(self) -> str:
        full_model = self.model_dump()
        return json.dumps(full_model['dataset_relationships'])

    @property
    def dataset_queries_json(self) -> str:
        full_model = self.model_dump()
        return json.dumps(full_model['dataset_queries'])

    @property
    def dataset_glossary_terms_json(self) -> str:
        full_model = self.model_dump()
        return json.dumps(full_model['dataset_business_glossary'])

    @property
    def text_table_ddls(self) -> str:
        table_ddls = '```\n'
        for table in self.dataset_tables:
            table_ddls += f"Table: {table.name}\n"
            table_ddls += f"DDL: {table.ddl}\n"

        table_ddls += '```'
        return table_ddls

### Helper Authentication

In [36]:
import requests, re

from google.cloud import bigquery
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import google.auth

class APIRequestError(Exception): pass
class AuthenticationError(APIRequestError): pass

class KEAuth:

    def __init__(self):
        self.__credentials = None
        self.__project = None

    def _get_credentials(self) -> Credentials:
            if self.__credentials is None:
                self.__credentials, self.__project = google.auth.default()

            if not self.__credentials.valid:
                try:
                    self.__credentials.refresh(Request())
                except Exception as e:
                    raise AuthenticationError(f"Failed to refresh Google credentials: {e}") from e

            return self.__credentials

    def _get_headers(self) -> dict:
        credentials = self._get_credentials()
        return {
          "Authorization": f"Bearer {credentials.token}",
          "Content-Type": "application/json"
        }

    def get_url_content(self, url: str) -> str:
            headers = self._get_headers()
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status() # Raises for 4xx or 5xx status codes
                return response.text

            except requests.exceptions.HTTPError as e:
                if e.response.status_code in (401, 403):
                    raise AuthenticationError(
                        f"Access Denied (HTTP {e.response.status_code}) fetching {url}. "
                        "Ensure your service account has the necessary IAM roles."
                    ) from e

                raise APIRequestError(f"HTTP Error {e.response.status_code} fetching {url}: {e.response.text}") from e

            except requests.exceptions.RequestException as e:
                raise APIRequestError(f"Network error fetching {url}: {e}") from e

### KEDatasetScanHelper

In [37]:

"""
  ------------------------------------------
  KEDatasetScanHelper
  ------------------------------------------
"""

class KEDatasetScanHelper(KEAuth):
    """A helper for interacting with the Knowledge Engine API."""
    DATAPLEX_BASE_URL = "https://dataplex.googleapis.com/v1"
    DATAPLEX_LIST_SCANS_URL = DATAPLEX_BASE_URL + "/projects/{project_id}/locations/{location}/dataScans"

    def __init__(self, project_id: str, dataset_name: str):
        super().__init__()
        self.dataset_name = dataset_name
        self.project_id = project_id
        self.__dataset_location = None
        self.__tables = []
        self.__data_scans = []
        self.__allowlist_tables = set()
        self.__blocklist_tables = set()
        self.__with_ddls = False
        self.__ddls = {}
        self.__with_table_counts = False
        self.__table_counts = {}

    def _flush(self):
        self.__tables.clear()
        self.__data_scans.clear()
        self.__ddls.clear()

    def _table_is_allowed(self, table_resource_fqn: str) -> bool:
        """
        Check if a table is allowed based on the allowlist and blocklist.
        The table resource FQN is in the format:
        //bigquery.googleapis.com/projects/{project_id}/datasets/{dataset_name}/tables/{table_name}
        """
        short_table_name = table_resource_fqn.split('/')[-1]

        return (
            self._is_in_allowlist(short_table_name) and not
            self._is_in_blocklist(short_table_name)
        )

    def _is_in_allowlist(self, short_table_name: str) -> bool:
        if not self.__allowlist_tables:
            return True

        return short_table_name in self.__allowlist_tables

    def _is_in_blocklist(self, short_table_name: str) -> bool:
        if not self.__blocklist_tables:
            return False

        return short_table_name in self.__blocklist_tables

    def _get_scans_of_interest(self) -> list:
        scan_url = self.DATAPLEX_LIST_SCANS_URL.format(
            base_url=self.DATAPLEX_BASE_URL,
            project_id=self.project_id,
            location=self.dataset_location
        )

        try:
            response = self.get_url_content(scan_url)
        except Exception as e:
            print(f"Error fetching data scans: {e}")
            raise e

        try:
            scans = json.loads(response)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response: {e}")
            raise e

        # Limit the scans to items in the requested dataset (per constructor)
        ds_test_string = f"/datasets/{self.dataset_name}"
        table_test_string = f"{ds_test_string}/tables/"

        scans_of_interest = []
        for scan in scans.get('dataScans', []):
            if scan.get('data') and scan.get('data').get('resource'):
                resource = scan.get('data').get('resource')

                if resource.endswith(ds_test_string) or table_test_string in resource:

                    try:
                      new_scan = DataScan(**scan)
                    except ValidationError as e:
                      print(f"Error creating DataScan object for {json.dumps(scan, indent=2)}:\n {e}")
                      raise e

                    if new_scan.is_for_table:
                        if self._table_is_allowed(new_scan.resource_name):
                            scans_of_interest.append(new_scan)

                    if new_scan.is_for_dataset:
                        scans_of_interest.append(new_scan)

        return scans_of_interest


    def with_table_list_constraints(self, allowlist: list = [], blocklist: list = []):
        """ configuration option """
        overlap = list(set(allowlist).intersection(set(blocklist)))
        if overlap:
            raise ValueError(f"Allowlist and blocklist cannot contain the same items: {overlap}")

        self._flush()
        self.__allowlist_tables.clear()
        self.__blocklist_tables.clear()
        self.__allowlist_tables.update(allowlist)
        self.__blocklist_tables.update(blocklist)

        return self

    def with_table_ddls(self, with_ddls=True):
        """ configuration option """
        self.__with_ddls = with_ddls
        self._flush()

        return self

    def with_table_counts(self, with_table_counts=True):
        """ configuration option """
        self.__with_table_counts = with_table_counts
        self._flush()

        return self

    @property
    def table_counts(self) -> dict:
        """ gets all the table counts for the dataset - row count, size_bytes"""
        if not self.__table_counts:
            client = bigquery.Client(project=self.project_id)
            query = f"""
                SELECT
                    CONCAT(project_id,'.',dataset_id,'.',table_id) AS fq_table_name
                , row_count
                , size_bytes
                FROM `{self.project_id}.{self.dataset_name}.__TABLES__`
            """
            query_job = client.query(query)
            results = query_job.result()

            for row in results:
                self.__table_counts[row.fq_table_name] = {
                    "row_count": row.row_count,
                    "size_bytes": row.size_bytes
                }

        return self.__table_counts

    @property
    def table_ddls(self) -> dict:
        """ gets all the table DDLs for the dataset """
        if not self.__ddls:
          client = bigquery.Client(project=self.project_id)
          query = f"""
              SELECT
                  CONCAT(
                      table_catalog,'.',table_schema,'.',table_name) AS fq_table_name,
                  ddl
              FROM `{self.project_id}.{self.dataset_name}.INFORMATION_SCHEMA.TABLES`
          """
          query_job = client.query(query)
          results = query_job.result()

          for row in results:
              self.__ddls[row.fq_table_name] = row.ddl

        return self.__ddls

    @property
    def dataset_location(self) -> str:
        if not self.__dataset_location:
            client = bigquery.Client()
            dataset = client.get_dataset(f'{self.project_id}.{self.dataset_name}')
            self.__dataset_location = dataset.location

        return self.__dataset_location

    @property
    def dataplex_scans(self) -> list:
        if not self.__data_scans:
            scans = self._get_scans_of_interest()

            for scan in scans:
                full_scan_url = f"{self.DATAPLEX_BASE_URL}/{scan.name}?view=FULL"

                try:
                    response = self.get_url_content(full_scan_url)
                except Exception as e:
                    print(f"Error fetching data scans: {e}")
                    raise e

                try:
                    full_view_scan = json.loads(response)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON response: {e}")
                    raise e

                new_scan = None

                if scan.type == ScanTypeValue.KNOWLEDGE_ENGINE:
                    new_scan = KEScan(**full_view_scan)

                if scan.type == ScanTypeValue.DATA_DOCUMENTATION:
                    if scan.is_for_table:
                        new_scan = DDTableScan(**full_view_scan)

                    if scan.is_for_dataset:
                        new_scan = DDDatasetScan(**full_view_scan)

                if new_scan:
                  self.__data_scans.append(new_scan)

        return self.__data_scans

    @property # dataset knowledge engine scan, loop locally
    def dataset_ke_scan(self) -> KEScan:
        for scan in self.dataplex_scans:
            if isinstance(scan, KEScan):
                return scan

    @property # dataset data documentation scan, loop locally
    def dataset_dd_scan(self) -> DDDatasetScan:
        for scan in self.dataplex_scans:
            if isinstance(scan, DDDatasetScan):
                return scan

    @property
    def dataset_description(self) -> str:
        return self.dataset_ke_scan.dataset_description

    @property
    def dataset_tables(self) -> List[KEDatasetTable]:
        tables = []
        for scan in self.dataplex_scans:
            if isinstance(scan, DDTableScan):
                if self._table_is_allowed(scan.resource_name):

                    ddl = None
                    if self.__with_ddls:
                        ddl = self.table_ddls.get(scan.full_table_name, None)

                    row_count = None
                    size_bytes = None
                    if self.__with_table_counts:
                        table_counts = self.table_counts.get(scan.full_table_name, None)
                        row_count = table_counts.get("row_count") if table_counts else None
                        size_bytes = table_counts.get("size_bytes") if table_counts else None

                    tables.append(KEDatasetTable(**{
                        "name": scan.full_table_name,
                        "overview": scan.overview,
                        "fields": scan.fields,
                        "queries": scan.queries,
                        "ddl": ddl,
                        "row_count": row_count,
                        "size_bytes": size_bytes,
                    }))

        return tables

    @property
    def dataset_queries(self) -> List[Query]:
        return self.dataset_dd_scan.queries

    @property
    def dataset_business_glossary(self) -> List[BusinessTerm]:
        return self.dataset_ke_scan.business_glossary.terms

    @property
    def dataset_relationships(self) -> List[KEDatasetRelationship]:
        """
          This will require update when the relation representation becomes more complex.
          Currently should handle multple anded = conditions between left and right side.
        """
        project_dataset = self.project_id + '.' + self.dataset_name

        return_relationships = []

        relationships = self.dataset_ke_scan.schema_relationships
        for relationship in relationships:

          left_tuples = relationship.left_columns_tuple
          table1_fqn = left_tuples[0].entry_fqn
          table1_sql_name = f"{project_dataset}.{table1_fqn.split('/')[-1]}"
          if not self._table_is_allowed(table1_fqn):
              continue

          right_tuples = relationship.right_columns_tuple
          table2_fqn = right_tuples[0].entry_fqn
          table2_sql_name = f"{project_dataset}.{table2_fqn.split('/')[-1]}"
          if not self._table_is_allowed(table2_fqn):
              continue

          join_conditions = []

          for i, left_item in enumerate(left_tuples):
              right_item = right_tuples[i]
              new_join_condition = table1_sql_name + '.' + left_item.field_path
              new_join_condition += ' = '
              new_join_condition += table2_sql_name + '.' + right_item.field_path
              join_conditions.append(new_join_condition)

          return_relationships.append(KEDatasetRelationship(**{
              'table1': table1_sql_name,
              'table2': table2_sql_name,
              'relationship': ' AND '.join(join_conditions),
              'source': 'LLM-inferred'
          }))

        return return_relationships

    @property
    def dataset_all_details(self) -> KEDatasetDetails:
        return KEDatasetDetails(**{
            "project_id": self.project_id,
            "dataset_name": self.dataset_name,
            "dataset_location": self.dataset_location,
            "dataset_description": self.dataset_description,
            "dataset_relationships": self.dataset_relationships,
            "dataset_queries": self.dataset_queries,
            "dataset_business_glossary": self.dataset_business_glossary,
            "dataset_tables": self.dataset_tables
        })

# Helper Testing

In [ ]:
PROJECT_ID = 'ai-learning-agents'
DATASET_NAME = 'thelook'
RESULTS_BUCKET = 'dataset_metadata_results'

ke_helper = KEDatasetScanHelper(PROJECT_ID, DATASET_NAME)
ke_helper.with_table_list_constraints(allowlist=[], blocklist=['sales'])
ke_helper.with_table_ddls()
ke_helper.with_table_counts()
ds_details = ke_helper.dataset_all_details

json_string = ds_details.model_dump_json()
print(json_string)

In [54]:
# Example Display with Mercury, change the
app = mr.App(title="Display notebook", static_notebook=True)
mr.JSON(json_string)

mercury.App

# Use for BQ Query Optimizer Agent

In [31]:
from google.cloud import storage

def write_json_to_gcs(bucket_name: str, json_data: str, file_name: str) -> None:
    """
    Writes a JSON string to a file in a Google Cloud Storage bucket.

    Args:
        bucket_name (str): The name of the GCS bucket.
        json_data (str): The JSON data as a string.
        file_name (str): The BigQuery dataset name.
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_name = f"{file_name}"
    blob = bucket.blob(blob_name)
    blob.upload_from_string(json_data, content_type="application/json")
    print(f"JSON data successfully written to gs://{bucket_name}/{blob_name}")

In [38]:
def make_query_optimizer_json(
    project_id,
    dataset_name,
    allowlist=[],
    blocklist=[]
) -> dict:
    """Create dict of only the stuff we want to use for Query Optimizer
        1. project_id
        2. dataset_name
        3. dataset_description
        4. dataset_location
        5. dataset_tables
            5.a name
            5.b overview
            5.c ddl
        6. dataset_relationships
    """
    ds_details = (
        KEDatasetScanHelper(PROJECT_ID, DATASET_NAME)
        .with_table_list_constraints(allowlist, blocklist)
        .with_table_ddls(True)
        .with_table_counts(True)
    ).dataset_all_details

    details_dict = json.loads(ds_details.model_dump_json())

    del details_dict['dataset_queries']
    del details_dict['dataset_business_glossary']

    for table in details_dict['dataset_tables']:
        del table['fields'] # they're in the ddl
        del table['queries']

    return json.dumps(details_dict)

In [39]:
### Write BQKE everything to gcs bucket

result_json = make_query_optimizer_json(PROJECT_ID, DATASET_NAME, blocklist=['sales'])
write_json_to_gcs(
    bucket_name=RESULTS_BUCKET,
    json_data=result_json,
    file_name=f"QUERY_OPTIMIZER_METADATA.{PROJECT_ID}.{DATASET_NAME}.json"
)

JSON data successfully written to gs://dataset_metadata_results/QUERY_OPTIMIZER_METADATA.ai-learning-agents.thelook.json


### Example Output

In [ ]:
"""
 {
  "project_id": "ai-learning-agents",
  "dataset_name": "thelook",
  "dataset_location": "us-central1",
  "dataset_description": "This dataset contains comprehensive information about e-commerce operations, encompassing user behavior, product details, order management, and inventory tracking. It provides a detailed view of customer interactions on the website, including browsing activity and order placements. The dataset includes granular information about products, such as pricing, category, and distribution center location. Order details are recorded, capturing the full lifecycle from creation to delivery and returns. Inventory levels and sales transactions are also tracked, allowing for analysis of product performance and supply chain efficiency. This data enables in-depth analysis of sales trends, user demographics, and operational effectiveness.",
  "dataset_relationships": [
    {
      "table1": "ai-learning-agents.thelook.order_items",
      "table2": "ai-learning-agents.thelook.orders",
      "relationship": "ai-learning-agents.thelook.order_items.order_id = ai-learning-agents.thelook.orders.order_id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.inventory_items",
      "table2": "ai-learning-agents.thelook.distribution_centers",
      "relationship": "ai-learning-agents.thelook.inventory_items.product_distribution_center_id = ai-learning-agents.thelook.distribution_centers.id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.order_items",
      "table2": "ai-learning-agents.thelook.inventory_items",
      "relationship": "ai-learning-agents.thelook.order_items.inventory_item_id = ai-learning-agents.thelook.inventory_items.id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.order_items",
      "table2": "ai-learning-agents.thelook.products",
      "relationship": "ai-learning-agents.thelook.order_items.product_id = ai-learning-agents.thelook.products.id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.orders",
      "table2": "ai-learning-agents.thelook.users",
      "relationship": "ai-learning-agents.thelook.orders.user_id = ai-learning-agents.thelook.users.id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.order_items",
      "table2": "ai-learning-agents.thelook.users",
      "relationship": "ai-learning-agents.thelook.order_items.user_id = ai-learning-agents.thelook.users.id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.inventory_items",
      "table2": "ai-learning-agents.thelook.products",
      "relationship": "ai-learning-agents.thelook.inventory_items.product_id = ai-learning-agents.thelook.products.id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.distribution_centers",
      "table2": "ai-learning-agents.thelook.products",
      "relationship": "ai-learning-agents.thelook.distribution_centers.id = ai-learning-agents.thelook.products.distribution_center_id",
      "source": "LLM-inferred"
    },
    {
      "table1": "ai-learning-agents.thelook.events",
      "table2": "ai-learning-agents.thelook.users",
      "relationship": "ai-learning-agents.thelook.events.user_id = ai-learning-agents.thelook.users.id",
      "source": "LLM-inferred"
    }
  ],
  "dataset_tables": [
    {
      "name": "ai-learning-agents.thelook.events",
      "overview": "This table stores records of user interactions and activities on a website or application. It captures various details associated with each event. The data includes information about the user, the event itself, and the context in which it occurred. This allows for analysis of user behavior, tracking of key performance indicators, and identification of trends. The table supports investigations into user journeys and event patterns.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.events`\n(\n  id INT64 OPTIONS(description=\"Unique identifier for the event.\"),\n  user_id INT64 OPTIONS(description=\"Unique identifier for the user.\"),\n  sequence_number INT64 OPTIONS(description=\"The order of the event within a user session.\"),\n  session_id STRING OPTIONS(description=\"Unique identifier for the user's session.\"),\n  created_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the event occurred.\"),\n  ip_address STRING OPTIONS(description=\"The IP address of the user who triggered the event.\"),\n  city STRING OPTIONS(description=\"The city from which the event originated.\"),\n  state STRING OPTIONS(description=\"The state or status associated with the event.\"),\n  postal_code STRING OPTIONS(description=\"The postal code associated with the event's location.\"),\n  browser STRING OPTIONS(description=\"The web browser used by the user.\"),\n  traffic_source STRING OPTIONS(description=\"The origin or source of the website traffic.\"),\n  uri STRING OPTIONS(description=\"The specific web address or resource being accessed.\"),\n  event_type STRING OPTIONS(description=\"The category or type of user event that occurred.\")\n)\nOPTIONS(\n  description=\"This table stores records of user activity on a website or application. It captures various events triggered by user interactions. The data includes details about the user's location, session, and browser. This information is used to understand user behavior and track website traffic.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-dp-published-scan\", \"ab20d79d1-33b8-4810-b06f-14a5280cbb00\"), (\"dataplex-dp-published-project\", \"ai-learning-agents\"), (\"dataplex-dp-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"a5f1f8396-cfad-4bc8-a25d-187c34201858\")]\n);",
      "row_count": 2425480,
      "size_bytes": 385142492
    },
    {
      "name": "ai-learning-agents.thelook.distribution_centers",
      "overview": "This table stores information about the locations of distribution centers. It provides a central repository for managing distribution center data. The table facilitates spatial analysis and mapping of these centers. It supports the identification and management of distribution center locations.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.distribution_centers`\n(\n  id INT64 OPTIONS(description=\"Unique identifier for the distribution center.\"),\n  name STRING OPTIONS(description=\"Name of the distribution center.\"),\n  latitude FLOAT64 OPTIONS(description=\"Latitude coordinate of the distribution center.\"),\n  longitude FLOAT64 OPTIONS(description=\"Longitude coordinate of the distribution center.\"),\n  distribution_center_geom GEOGRAPHY OPTIONS(description=\"Geographical location of the distribution center.\")\n)\nOPTIONS(\n  description=\"This table stores information about distribution centers. It includes location data for each center. The data allows for spatial analysis and mapping. It also supports identifying and managing distribution center locations.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"add9d8061-6552-4d49-81bc-2bc3943a5b79\"), (\"dataplex-dp-published-project\", \"ai-learning-agents\"), (\"dataplex-dp-published-location\", \"us-central1\"), (\"dataplex-dp-published-scan\", \"a2886da71-0f35-4192-8742-bd9b1ffc4ddd\")]\n);",
      "row_count": 10,
      "size_bytes": 809
    },
    {
      "name": "ai-learning-agents.thelook.inventory_items",
      "overview": "This table tracks the inventory of products. It records details about each item, including its cost and retail price. The table also associates each item with a specific product and distribution center. This allows for analysis of inventory levels and product performance.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.inventory_items`\n(\n  id INT64 OPTIONS(description=\"Unique identifier for the inventory item.\"),\n  product_id INT64 OPTIONS(description=\"Identifier for the product.\"),\n  created_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the inventory item was created.\"),\n  sold_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the inventory item was sold.\"),\n  cost FLOAT64 OPTIONS(description=\"Cost of the inventory item.\"),\n  product_category STRING OPTIONS(description=\"Category of the product.\"),\n  product_name STRING OPTIONS(description=\"Name of the product.\"),\n  product_brand STRING OPTIONS(description=\"Brand of the product.\"),\n  product_retail_price FLOAT64 OPTIONS(description=\"Retail price of the product.\"),\n  product_department STRING OPTIONS(description=\"Department the product belongs to.\"),\n  product_sku STRING OPTIONS(description=\"Stock keeping unit for the product.\"),\n  product_distribution_center_id INT64 OPTIONS(description=\"Identifier for the distribution center where the product is located.\")\n)\nOPTIONS(\n  description=\"This table tracks the inventory of products. It records details about each item, including its cost and retail price. The table also associates each item with a specific product and distribution center. This allows for analysis of inventory levels and product performance.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"ada6bf718-997a-49c9-9c39-420e79abe1cc\")]\n);",
      "row_count": 489838,
      "size_bytes": 81275603
    },
    {
      "name": "ai-learning-agents.thelook.orders",
      "overview": "This table stores information about customer orders. It tracks the status of each order throughout its lifecycle. The table records timestamps for when an order was created, shipped, delivered, and potentially returned. It also includes data related to the user who placed the order.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.orders`\n(\n  order_id INT64 OPTIONS(description=\"Unique identifier for the order.\"),\n  user_id INT64 OPTIONS(description=\"Unique identifier for the user who placed the order.\"),\n  status STRING OPTIONS(description=\"Status of the order.\"),\n  gender STRING OPTIONS(description=\"Gender of the user who placed the order.\"),\n  created_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order was created.\"),\n  returned_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order was returned.\"),\n  shipped_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order was shipped.\"),\n  delivered_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order was delivered.\"),\n  num_of_item INT64 OPTIONS(description=\"Number of items in the order.\")\n)\nOPTIONS(\n  description=\"This table stores information about customer orders. It tracks the status of each order throughout its lifecycle. The table records timestamps for when an order was created, shipped, delivered, and potentially returned. It also includes data related to the user who placed the order.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"ab06ef464-2420-4997-b3c0-7cfa94bdad28\")]\n);",
      "row_count": 125105,
      "size_bytes": 6762580
    },
    {
      "name": "ai-learning-agents.thelook.users",
      "overview": "This table stores user information. It includes demographic details and geographic location. The table also tracks the source from which users originate. This data can be used for user segmentation and targeted marketing efforts.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.users`\n(\n  id INT64 OPTIONS(description=\"Unique identifier for the user.\"),\n  first_name STRING OPTIONS(description=\"First name of the user.\"),\n  last_name STRING OPTIONS(description=\"Last name of the user.\"),\n  email STRING OPTIONS(description=\"Email address of the user.\"),\n  age INT64 OPTIONS(description=\"Age of the user.\"),\n  gender STRING OPTIONS(description=\"Gender of the user.\"),\n  state STRING OPTIONS(description=\"State of the user.\"),\n  street_address STRING OPTIONS(description=\"Street address of the user.\"),\n  postal_code STRING OPTIONS(description=\"Postal code of the user's address.\"),\n  city STRING OPTIONS(description=\"City of the user.\"),\n  country STRING OPTIONS(description=\"Country of the user.\"),\n  latitude FLOAT64 OPTIONS(description=\"Latitude coordinate of the user's location.\"),\n  longitude FLOAT64 OPTIONS(description=\"Longitude coordinate of the user's location.\"),\n  traffic_source STRING OPTIONS(description=\"Source from which the user originated.\"),\n  created_at TIMESTAMP OPTIONS(description=\"Timestamp when the user account was created.\"),\n  user_geom GEOGRAPHY OPTIONS(description=\"Geographic data representing the user's location.\")\n)\nOPTIONS(\n  description=\"This table stores user information. It includes demographic details and geographic location. The table also tracks the source from which users originate. This data can be used for user segmentation and targeted marketing efforts.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-dp-published-scan\", \"ac26a7027-1ec9-4ebe-a5e6-c0a41ab6108d\"), (\"dataplex-dp-published-project\", \"ai-learning-agents\"), (\"dataplex-dp-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"a62faff99-c9a5-4687-a4fa-8687859916ec\")]\n);",
      "row_count": 100000,
      "size_bytes": 19813856
    },
    {
      "name": "ai-learning-agents.thelook.users_view",
      "overview": "This table stores user information. It includes details that can be used for user segmentation and demographic analysis. The data allows for geographic analysis of users. The table also tracks the source from which users originated. This information supports marketing and customer relationship management efforts.",
      "ddl": null,
      "row_count": null,
      "size_bytes": null
    },
    {
      "name": "ai-learning-agents.thelook.products",
      "overview": "This table stores comprehensive details about products. It includes information on product categorization and branding. The table also contains pricing and cost data. This data can be used for inventory management and sales analysis.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.products`\n(\n  id INT64 OPTIONS(description=\"Unique identifier for the product.\"),\n  cost FLOAT64 OPTIONS(description=\"Cost of the product.\"),\n  category STRING OPTIONS(description=\"Category that the product belongs to.\"),\n  name STRING OPTIONS(description=\"Name of the product.\"),\n  brand STRING OPTIONS(description=\"Brand of the product.\"),\n  retail_price FLOAT64 OPTIONS(description=\"Retail price of the product.\"),\n  department STRING OPTIONS(description=\"Department that the product belongs to.\"),\n  sku STRING OPTIONS(description=\"Stock keeping unit of the product.\"),\n  distribution_center_id INT64 OPTIONS(description=\"Unique identifier for the distribution center where the product is stocked.\")\n)\nOPTIONS(\n  description=\"This table stores comprehensive details about products. It includes information on product categorization and branding. The table also contains pricing and cost data. This data can be used for inventory management and sales analysis.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"af1451be0-1392-405e-a88b-bc58600cb0e7\"), (\"dataplex-dp-published-project\", \"ai-learning-agents\"), (\"dataplex-dp-published-location\", \"us-central1\"), (\"dataplex-dp-published-scan\", \"ab58cf10a-1ccc-40eb-95b0-563f48b162e3\")]\n);",
      "row_count": 29120,
      "size_bytes": 4285975
    },
    {
      "name": "ai-learning-agents.thelook.order_items",
      "overview": "This table tracks individual items within customer orders. It records the timestamps for when each item was created, shipped, delivered, and potentially returned. The table also stores the sale price for each item. It provides a comprehensive view of the lifecycle of each item in an order.",
      "ddl": "CREATE TABLE `ai-learning-agents.thelook.order_items`\n(\n  id INT64 OPTIONS(description=\"Unique identifier for the order item.\"),\n  order_id INT64 OPTIONS(description=\"Identifier for the order to which the item belongs.\"),\n  user_id INT64 OPTIONS(description=\"Identifier for the user who placed the order.\"),\n  product_id INT64 OPTIONS(description=\"Identifier for the product associated with the order item.\"),\n  inventory_item_id INT64 OPTIONS(description=\"Identifier for the inventory item associated with the order item.\"),\n  status STRING OPTIONS(description=\"Status of the order item.\"),\n  created_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order item was created.\"),\n  shipped_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order item was shipped.\"),\n  delivered_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order item was delivered.\"),\n  returned_at TIMESTAMP OPTIONS(description=\"Timestamp indicating when the order item was returned.\"),\n  sale_price FLOAT64 OPTIONS(description=\"The price at which the order item was sold.\")\n)\nOPTIONS(\n  description=\"This table stores data about individual items within customer orders. It tracks when each item was ordered, shipped, and delivered. The table also includes pricing information for each item. It provides a record of the status of each item in an order.\",\n  labels=[(\"dataplex-data-documentation-published-location\", \"us-central1\"), (\"dataplex-dp-published-scan\", \"aedb784a7-c153-4f58-b89c-a2a9da201127\"), (\"dataplex-dp-published-project\", \"ai-learning-agents\"), (\"dataplex-dp-published-location\", \"us-central1\"), (\"dataplex-data-documentation-published-project\", \"ai-learning-agents\"), (\"dataplex-data-documentation-published-scan\", \"a86a52b65-617e-4358-96b8-12353643d393\")]\n);",
      "row_count": 181396,
      "size_bytes": 13614998
    }
  ]
}
"""

### Agent Prompt

In [ ]:
"""
You are a Google BigQuery Expert and you are highly skilled at evaluating and optimizing queries that users provide to you.
Users provide queries to you, and you use all relevant information available to you to analyze the query and identify ways to make
  more cost performant by minimizing the amount of data scanned and the amount of data processed.
When a user provides a query to you, you will identify the PROJECT_ID and DATASET_NAME the query is being run on.
If that information is not obvious from the query itself, then interact with the user to try to get that information.

You will use that information in an attempt to access metadata about the dataset that the query is being run on.

You will have access to a gcs storage bucket in which metadata about the dataset may exist.
The storage bucket is "gs://dataset_metadata_results"
Metadata files are named according to the following naming convention:
    - QUERY_OPTIMIZER_METADATA.{PROJECT_ID}.{DATASET_NAME}.json

Be helpful in the user interaction where you attempt to identifying PROJECT_ID and DATASET_NAME.
You are able to suggest combinations by scanning the contents of the storage bucket and listing PROJECT_ID and DATASET_NAME pairs you have metadata for.

If a matching metadata file *does not exist* in the bucket, use your best judgement to optimize the query.
If a matching metadata file *does exist* in the bucket, use that metadata to optimize the query.

If metadata exists, it will be in JSON format and will contains a wealth of information about the dataset, including:
    - The name of the dataset
    - The description of the dataset
    - The location of the dataset
    - A list of tables in the dataset
    - The relationships between the tables in the dataset

For each of the tables, you will get the following information:
    - The name of the table
    - The description of the table
    - The ddl for the table

Pay special attention to any table ddls that are provided.
The table ddls will contain:
    - Column names
    - Column data types
    - Column comments (e.g. column descriptions)
    - Column constraints (e.g. NOT NULL, PRIMARY KEY, FOREIGN KEY)
    - Column references (e.g. foreign key constraints)
    - Table constraints (e.g. UNIQUE, PRIMARY KEY)
    - Table references (e.g. foreign key constraints)
    - Table comments (e.g. table descriptions)
    - Table optimization options (e.g. PARTITIONED BY, CLUSTERED BY)
    - Table row count
    - Table size in bytes

Leverage your bigquery expertise along with table optimization options appearing in the ddls (like Partitioning and Clustering) to suggest optimizations.
Provide helpful commentary on the query optimizations that you suggest.
Look for opportunities to suggest dataset level optimizations based on the types of queries the user asks about (for example, if partitioning, or clustering would be helpful).

Important: Always ground your answer in metadata that is available to you in the storage bucket, or with information provided by the user.
"""